In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

In [2]:
def get_spb_districts_from_osm() -> gpd.GeoDataFrame:
    districts = [
        "Приморский район", 
        "Московский район",
        "Курортный район",
        "Центральный район",
        "Пушкинский район",
        "Фрунзенский район",
        "Василеостровский район",
        "Колпинский район",
        "Петроградский район",
        "Кронштадтский район",
        "Красносельский район",
        "Кировский район",
        "Петродворцовый район",
        "Адмиралтейский район",
        "Невский район",
        "Калининский район",
        "Красногвардейский район",
        "Выборгский район"
    ]
    spb_districts: gpd.GeoDataFrame = ox.features_from_place("Saint-Petersburg", tags={"name":districts})
    spb_districts = spb_districts.dropna(subset=['addr:region'])
    spb_districts = spb_districts[spb_districts['addr:region'] != 'Ленинградская область']
    spb_districts = spb_districts.reset_index()[['geometry', "name"]]
    return spb_districts
spb_districts = get_spb_districts_from_osm()
spb_districts

,geometry,name
0,"POLYGON ((30.44112 59.82937, 30.44333 59.82980...",Колпинский район
1,"POLYGON ((30.21822 59.67307, 30.21918 59.67361...",Пушкинский район
2,"POLYGON ((30.19877 59.80161, 30.19901 59.80180...",Московский район
3,"MULTIPOLYGON (((30.06506 59.66830, 30.07331 59...",Красносельский район
4,"POLYGON ((30.36331 59.91387, 30.36951 59.91412...",Невский район
5,"POLYGON ((30.33258 59.91305, 30.33352 59.91337...",Фрунзенский район
6,"POLYGON ((30.25023 59.90129, 30.25028 59.90132...",Адмиралтейский район
7,"POLYGON ((30.18468 59.95472, 30.18473 59.95481...",Василеостровский район
8,"POLYGON ((30.16799 59.88702, 30.16817 59.88725...",Кировский район
9,"POLYGON ((30.39591 59.92574, 30.39428 59.92902...",Красногвардейский район


In [3]:
def get_spb_food_places_from_osm() -> gpd.GeoDataFrame:
    tags = {
        "amenity": ["bar", 	"biergarten", "cafe", "fast_food", "food_court", "ice_cream", "pub", "restaurant"]
    }
    needed_columns = ["geometry", "opening_hours", "cuisine", "amenity", "food", "delivery", "name"]
    data = ox.features_from_place("Saint-Petersburg", tags=tags).reset_index()[needed_columns]
    return data
spb_food_places = get_spb_food_places_from_osm()
spb_food_places

,geometry,opening_hours,cuisine,amenity,food,delivery,name
0,POINT (30.31320 59.91540),13:00-02:00,NaN,bar,NaN,NaN,Контакт Бар
1,POINT (30.28913 59.92477),09:00-22:00,lebanese,cafe,NaN,NaN,Wonderful Taste
2,POINT (30.37544 59.91366),Mo-Fr 10:00-17:00,NaN,cafe,NaN,NaN,Динер холл
3,POINT (30.37681 59.91408),NaN,NaN,cafe,NaN,NaN,Амилен
4,POINT (30.36320 59.93859),12:00-02:00,NaN,pub,NaN,NaN,The Templet Bar
...,...,...,...,...,...,...,...
8377,"POLYGON ((29.51854 60.19336, 29.51858 60.19337...",NaN,kebab,fast_food,NaN,NaN,Шаверма
8378,"POLYGON ((29.70319 60.20569, 29.70317 60.20565...",07:00-05:00,NaN,fast_food,NaN,NaN,Просто Вася
8379,"POLYGON ((29.96952 60.07136, 29.96968 60.07137...",Mo-Su 08:00-23:00,burger,fast_food,NaN,NaN,Бургер Кинг
8380,"POLYGON ((29.85248 60.16144, 29.85234 60.16144...",Mo-Su 12:00-24:00,regional,restaurant,NaN,NaN,Панорама


In [4]:
map = KeplerGl(data={"food_places": spb_food_places, "districts": spb_districts})
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


d:\учеба\trkpo_service\trkpo_venv\lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'food_places':                                                geometry      opening_hours  \
0 …

In [5]:
def spatial_join_food_places_and_districts(food_places: gpd.GeoDataFrame, districts: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    return gpd.sjoin(food_places, districts).rename(columns={'name_left': 'name', 'name_right': 'district'}).drop(columns=['index_right'])
food_places_and_districts = spatial_join_food_places_and_districts(spb_food_places, spb_districts)
food_places_and_districts

,geometry,opening_hours,cuisine,amenity,food,delivery,name,district
0,POINT (30.31320 59.91540),13:00-02:00,NaN,bar,NaN,NaN,Контакт Бар,Адмиралтейский район
1,POINT (30.28913 59.92477),09:00-22:00,lebanese,cafe,NaN,NaN,Wonderful Taste,Адмиралтейский район
25,POINT (30.29158 59.93156),"Mo-Th,Su 12:00-21:00; Fr-Sa 12:00-22:00",NaN,cafe,NaN,NaN,NowaDays,Адмиралтейский район
26,POINT (30.29681 59.91807),12:00-22:30,chinese,restaurant,NaN,NaN,Тайвань,Адмиралтейский район
29,POINT (30.33930 59.91947),07:00-23:30,burger,fast_food,NaN,NaN,Вкусно — и точка,Адмиралтейский район
...,...,...,...,...,...,...,...,...
8345,POINT (29.74450 59.99570),NaN,italian,restaurant,NaN,NaN,Vento di Coda,Кронштадтский район
8358,"POLYGON ((29.75294 60.00376, 29.75312 60.00371...",NaN,NaN,restaurant,NaN,NaN,Brew fort,Кронштадтский район
8364,"POLYGON ((29.70643 59.99446, 29.70639 59.99434...",NaN,NaN,cafe,NaN,NaN,Летнее кафе,Кронштадтский район
8372,"POLYGON ((29.77139 59.98550, 29.77153 59.98548...","Mo-Fr 11:00-20:00; Sa,Su 11:00-22:00",NaN,cafe,NaN,NaN,Вкусное мясо,Кронштадтский район


1. Улица - находим Polygon в указанном районе, где нету заведений с указанным типом кухни и типом заведения. В этом полигоне берем несколько объявлений циан 
2. Точный адрес помещения - берем из объявления циан в пункте 1
3. Площадь помещения (м2) - берем из объявления циан в пункте 1
4. Цена за помещение (руб/мес) - берем из объявления циан в пункте 1
5. Выгодное время работы (временной промежуток) - берем все заведения в районе (неважно какой тип, кухня) и смотрим самое раннее открытие N и самое позднее закрытие M. Возьмем для простоты выгодное время работы как [N-1ч, M+2ч].
6. Возможность доставки (по возможности доставки соседних заведений) - берем все заведения в районе (неважно какой тип, кухня). Если больше, чем 50% имеют доставку, то не будем рекомендовать ее. Если меньше или равно, то будем рекомендовать.

